# Composition of observation spaces for corn environment
In this notebook, we show how to compose different observation spaces for a one-year corn environment

In [1]:
from cyclesgym.envs import Corn
import numpy as np

First, we show how to initialize the standard one-year corn environment and print the name of its observation.

In [2]:
delta = 7 # Time discretization of 7 days
n_actions = 11 # 11 discrete actions
maxN = 150 # actions equally spaced in [0, 150]

# Initialize a single year corn environment 
base_corn_env = Corn(delta=delta, n_actions=n_actions, maxN=maxN)  

# Get an observation (needed to initialize the observer)
s = base_corn_env.reset()

# Print observation names
base_corn_env.observer.obs_names


Simulation 2022_05_25_22_29_40-d413f632-e607-417b-aa99-448e6ae154e3/control running ...

Simulation time: 0 seconds.


/Users/scottsussex/Documents/ETH/agroscope/cyclesgym/cyclesgym/managers/operation.py:93: UserWarning: There is already an operation FIXED_FERTILIZATION for they day 110 of year 1.
  warnings.warn(f'There is already an operation {operation} for they day {doy} of year {year}.')


['LATITUDE',
 'ALTITUDE',
 'SCREENING_HEIGHT',
 'PP',
 'TX',
 'TN',
 'SOLAR',
 'RHX',
 'RHN',
 'WIND',
 'STAGE',
 'THERMAL TIME',
 'CUM. BIOMASS',
 'AG BIOMASS',
 'ROOT BIOMASS',
 'FRAC INTERCEP',
 'TOTAL N',
 'AG N',
 'ROOT N',
 'AG N CONCN',
 'N FIXATION',
 'N ADDED',
 'N STRESS',
 'WATER STRESS',
 'POTENTIAL TR',
 'DOY',
 'N TO DATE']

Now, we show how to override a the standard corn env to obtain an environment with more complex observations. To this end, we change __init__ to make Cycles write the Nitrogen profile as output. Then, we initialize the corresponding manager. Finally, we use that manager to define a compound observation space.

In [3]:
import cyclesgym.envs.observers as observers
from cyclesgym.managers import SoilNManager
from cyclesgym.envs.common import CyclesEnv

class CornSoilCropWeatherObs(Corn):
    # Need to write N to output
    def __init__(self,
                 delta,
                 n_actions,
                 maxN,
                 operation_file='ContinuousCorn.operation',
                 soil_file='GenericHagerstown.soil',
                 weather_file='RockSprings.weather',
                 start_year=1980,
                 end_year=1980,
                 use_reinit=True
                 ):
        self.rotation_size = end_year - start_year + 1
        self.use_reinit = use_reinit
        CyclesEnv.__init__(self, 
                          SIMULATION_START_YEAR=1980,
                         SIMULATION_END_YEAR=1980,
                         ROTATION_SIZE=1,
                         USE_REINITIALIZATION=0,
                         ADJUSTED_YIELDS=0,
                         HOURLY_INFILTRATION=1,
                         AUTOMATIC_NITROGEN=0,
                         AUTOMATIC_PHOSPHORUS=0,
                         AUTOMATIC_SULFUR=0,
                         DAILY_WEATHER_OUT=0,
                         DAILY_CROP_OUT=1,
                         DAILY_RESIDUE_OUT=0,
                         DAILY_WATER_OUT=0,
                         DAILY_NITROGEN_OUT=1,  
                         DAILY_SOIL_CARBON_OUT=0,
                         DAILY_SOIL_LYR_CN_OUT=0,
                         ANNUAL_SOIL_OUT=0,
                         ANNUAL_PROFILE_OUT=0,
                         ANNUAL_NFLUX_OUT=0,
                         CROP_FILE='GenericCrops.crop',
                         OPERATION_FILE=operation_file,
                         SOIL_FILE=soil_file,
                         WEATHER_FILE=weather_file,
                         REINIT_FILE='N / A',
                         delta=delta)
        self._post_init_setup()
        self._init_observer()
        self._generate_observation_space()
        self._generate_action_space(n_actions, maxN)
    
    # Add N manager to fields
    def _post_init_setup(self):
        super()._post_init_setup()
        self.soil_n_file = None
        self.soil_n_manager = None
    
    # Initialize soil N manager
    def _init_output_managers(self):
        super()._init_output_managers()
        self.soil_n_file = self._get_output_dir().joinpath('N.dat')
        self.soil_n_manager = SoilNManager(self.soil_n_file)
    
    # Add observer of soild to compoud one
    def _init_observer(self, *args, **kwargs):
        end_year = self.ctrl_base_manager.ctrl_dict['SIMULATION_END_YEAR']
        self.observer = observers.compound_observer([
            observers.WeatherObserver(weather_manager=self.weather_manager, end_year=end_year),
            observers.CropObserver(crop_manager=self.crop_output_manager, end_year=end_year),
            observers.SoilNObserver(soil_n_manager=self.soil_n_manager, end_year=end_year),
            observers.NToDateObserver(end_year=end_year)
                                           ])
        
    

Now we take a look at the observation space of our newly created environment

In [4]:
large_obs_corn_env = CornSoilCropWeatherObs(delta=delta, n_actions=n_actions, maxN=maxN)
s = large_obs_corn_env.reset()
large_obs_corn_env.observer.obs_names


Simulation 2022_05_25_22_31_30-0c5d1df6-1b30-4bf5-9ed3-6c75fb2516e8/control running ...

Simulation time: 0 seconds.


/Users/scottsussex/Documents/ETH/agroscope/cyclesgym/cyclesgym/managers/operation.py:93: UserWarning: There is already an operation FIXED_FERTILIZATION for they day 110 of year 1.
  warnings.warn(f'There is already an operation {operation} for they day {doy} of year {year}.')


['LATITUDE',
 'ALTITUDE',
 'SCREENING_HEIGHT',
 'PP',
 'TX',
 'TN',
 'SOLAR',
 'RHX',
 'RHN',
 'WIND',
 'STAGE',
 'THERMAL TIME',
 'CUM. BIOMASS',
 'AG BIOMASS',
 'ROOT BIOMASS',
 'FRAC INTERCEP',
 'TOTAL N',
 'AG N',
 'ROOT N',
 'AG N CONCN',
 'N FIXATION',
 'N ADDED',
 'N STRESS',
 'WATER STRESS',
 'POTENTIAL TR',
 'ORG SOIL N',
 'PROF SOIL NO3',
 'PROF SOIL NH4',
 'MINERALIZATION',
 'IMMOBILIZATION',
 'NET MINERALIZ',
 'NH4 NITRIFICAT',
 'N2O FROM NITRIF',
 'NH3 VOLATILIZ',
 'NO3 DENITRIF',
 'N2O FROM DENIT',
 'DOY',
 'N TO DATE']

Finally, we use the PartialObsEnv wrapper to get rid of the observations we do not want to use.

In [5]:
from cyclesgym.envs.common import PartialObsEnv
target_obs = ['PP', # Precipitation
              'TX', # Max temperature
              'TN', # Min temperature
              'SOLAR', # Radiation
              'RHX', # Max relative humidity
              'RHN', # Min relative humidity
              'STAGE', # Stage in the plant life cycle
              'CUM. BIOMASS', # Cumulative plant biomass
              'N STRESS', 
              'WATER STRESS',
              'ORG SOIL N', # The sum of microbial biomass N and stabilized soil organic N pools.
              'PROF SOIL NO3', # Soil profile nitrate-N content.
              'PROF SOIL NH4' # Soil profile ammonium-N content.
             ]
mask = np.isin(np.asarray(large_obs_corn_env.observer.obs_names), target_obs)
smart_obs_corn_env = PartialObsEnv(CornSoilCropWeatherObs(delta=delta, n_actions=n_actions, maxN=maxN), mask=mask)
smart_obs_corn_env.reset()
smart_obs_corn_env.observer.obs_names


Simulation 2022_05_25_22_31_35-a049dc97-e766-403e-bb61-1c03b96aaa66/control running ...

Simulation time: 0 seconds.


/Users/scottsussex/Documents/ETH/agroscope/cyclesgym/cyclesgym/managers/operation.py:93: UserWarning: There is already an operation FIXED_FERTILIZATION for they day 110 of year 1.
  warnings.warn(f'There is already an operation {operation} for they day {doy} of year {year}.')


['PP',
 'TX',
 'TN',
 'SOLAR',
 'RHX',
 'RHN',
 'STAGE',
 'CUM. BIOMASS',
 'N STRESS',
 'WATER STRESS',
 'ORG SOIL N',
 'PROF SOIL NO3',
 'PROF SOIL NH4']